In [0]:
df_bronze = spark.table("bronze.flights.us_flights")
df_bronze.display()

In [0]:
df_silver = df_bronze

df_silver = df_silver.drop("tbl", "tbl1apk", "Geocoded_City1", "Geocoded_City2")

rename_map = {
    "Year": "year",
    "airportid_1": "airportid_origin",
    "airportid_2": "airportid_dest",
    "airport_1": "airport_origin",
    "airport_2": "airport_dest",
    "nsmiles": "distance_miles",
    "citymarketid_1": "citymarket_origin",
    "citymarketid_2": "citymarket_dest",
    "city1": "city_origin",
    "city2": "city_dest",
    "fare": "avg_fare",
    "fare_lg": "avg_fare_large_carry",
    "large_ms": "market_share_large_carry",
    "fare_low": "avg_fare_low_cost",
    "lf_ms": "market_share_low_cost",
    "carrier_lg": "largest_carry_by_passengers",
    "carrier_low": "lowest_carry_by_passengers"
}

for old_col, new_col in rename_map.items():
    df_silver = df_silver.withColumnRenamed(old_col, new_col)

In [0]:
df_silver.display()

In [0]:
# Write to the table
df_silver.coalesce(1).write.mode("overwrite").saveAsTable("silver.flights.flights")